###### Content under Creative Commons Attribution license CC-BY 4.0, code under MIT license (c) 2019 Daniel Koehn, based on (c)2018 L.A. Barba, G.F. Forsyth [CFD Python](https://github.com/barbagroup/CFDPython#cfd-python), (c)2014 L.A. Barba, I. Hawke, B. Knaepen [Practical Numerical Methods with Python](https://github.com/numerical-mooc/numerical-mooc#practical-numerical-methods-with-python), also under CC-BY.

In [1]:
# Execute this cell to load the notebook's style sheet, then ignore it
from IPython.core.display import HTML
css_file = '../style/custom.css'
HTML(open(css_file, "r").read())

# 2D Heat Equation: Comparison between analytical and numerical solution

In this module, we developed a finite difference modelling code to solve the 2D heat equation and optimized the runtime performance using Just-In-Time (JIT) compilation from the `Numba` package. What is still missing is a comparison between an analytical and the corresponding numerical solution.

## Analytical solution of the 2D Heat Equation

A simple (time-dependent) analytical solution for the 2D heat equation 

\begin{equation}
\frac{\partial T}{\partial t} = \alpha \left(\frac{\partial^2 T}{\partial x^2} + \frac{\partial^2 T}{\partial y^2} \right) \tag{1}
\end{equation}

exists for the case that the initial temperature distribution in a full-space model

\begin{equation}
T(x,y,t=0) = T_{max} exp\biggl[\frac{-(x^2+y^2)}{s^2}\biggr]\tag{2}
\end{equation}

where $T_{max}$ is the maximum amplitude of the temperature perturbation at $(x,y)=(0,0)$ and it 's half-width $s$. Then, the analytical solution is

\begin{equation}
T(x,y,t) = \frac{T_{max}}{1+4t\alpha/s^2} exp\biggl[\frac{-(x^2+y^2)}{s^2+4t\alpha}\biggr]\tag{3}
\end{equation}

##### Exercise 1

Solve the above problem using the JIT-optimized FTCS FD-code from the last class and compare the numerical with the analytical solution.

Let's start by setting up our Python compute environment and reuse the performance optimized `ftcs_JIT` code from the last class.

In [ ]:
# Import libraries
import numpy
from matplotlib import pyplot
%matplotlib inline

# import JIT from Numba
from numba import jit

In [ ]:
# Set the font family and size to use for Matplotlib figures.
pyplot.rcParams['font.family'] = 'serif'
pyplot.rcParams['font.size'] = 16

In [ ]:
# FTCS code to solve the 2D heat equation with JIT optimization
# -------------------------------------------------------------
@jit(nopython=True) # use Just-In-Time (JIT) Compilation for C-performance
def ftcs_JIT(T0, nt, dt, dx, dy, alpha):
    """
    Computes and returns the temperature distribution
    after a given number of time steps.
    Explicit integration using forward differencing
    in time and central differencing in space, with
    Neumann conditions (zero-gradient) on top and right
    boundaries and Dirichlet conditions on bottom and
    left boundaries.
    
    Parameters
    ----------
    T0 : numpy.ndarray
        The initial temperature distribution as a 2D array of floats.
    nt : integer
        Maximum number of time steps to compute.
    dt : float
        Time-step size.
    dx : float
        Grid spacing in the x direction.
    dy : float
        Grid spacing in the y direction.
    alpha : float
        Thermal diffusivity.
    
    Returns
    -------
    T : numpy.ndarray
        The temperature distribution as a 2D array of floats.
    """
    # Define some constants.
    sigma_x = alpha * dt / dx**2
    sigma_y = alpha * dt / dy**2
    
    # Integrate in time.
    T = T0.copy()
    
    # Estimate number of grid points in x- and y-direction
    ny, nx = T.shape
    
    # Indices of the model center
    I, J = int(nx / 2), int(ny / 2)  
    
    # Time loop
    for n in range(nt):
        
        # store old temperature field
        Tn = T.copy()
        
        # loop over spatial grid 
        for i in range(1,nx-1):
            for j in range(1,ny-1):
                
                T[j, i] = (Tn[j, i] +
                         sigma_x * (Tn[j, i+1] - 2.0 * Tn[j, i] + Tn[j, i-1]) +
                         sigma_y * (Tn[j+1, i] - 2.0 * Tn[j, i] + Tn[j-1, i]))                
        
    return T

Define modelling parameters and initial conditions according to eq. (2). This time, I give you the freedom to define your own model parameters

In [ ]:
# Definition of modelling parameters
# ----------------------------------
Lx =   # length of the plate in the x direction [m]
Ly =   # height of the plate in the y direction [m]
nx =   # number of points in the x direction
ny =   # number of points in the y direction
dx = Lx / (nx - 1)  # grid spacing in the x direction
dy = Ly / (ny - 1)  # grid spacing in the y direction
alpha =   # thermal diffusivity of the plate [m^2/s]

# Define the locations along a gridline.
x = numpy.linspace(0.0, Lx, num=nx)
y = numpy.linspace(0.0, Ly, num=ny)

# DEFINE THE INITIAL TEMPERATURE DISTRIBUTION EQ.(2) HERE!
X, Y = numpy.meshgrid(x,y) # coordinates X,Y required to define T0

# I recommend moving the maximum of the initial temperature 
# distribution to the center of the model
X = X - Lx/2.
Y = Y - Ly/2.

s =         # half-width of the Gaussian function [m]
Tmax =      # maximum temperature Tmax [°C]
T0 =        # Define initial temperature distribution according to eq.(2)

We don't want our solution blowing up, so let's find a time step with $\frac{\alpha \Delta t}{\Delta x^2} = \frac{\alpha \Delta t}{\Delta y^2} = \frac{1}{4}$. Also, define the number of time steps `nt` you want to model the heat conduction

In [ ]:
# Set the time-step size based on CFL limit.
sigma = 0.25
dt = sigma * min(dx, dy)**2 / alpha  # time-step size
nt =   # number of time steps to compute

After setting all modelling parameters, we can run the `ftcs_JIT` modelling code to compute the numerical solution after `nt` time steps 

In [ ]:
# Compute the temperature distribution after nt timesteps
T = ftcs_JIT(T0, nt, dt, dx, dy, alpha)

Compute the temperature field of the analytical solution

In [ ]:
# DEFINE ANALYTICAL SOLUTION EQ. (3) HERE!
t = nt * dt  # maximum modelling time of the FD code
T_analytical = 

In order to compare the different solutions, we first plot the analytical temperature distribution. Depending on how you defined the problem, you might have to adjust the temperature range in the `levels` array

In [ ]:
# Plot the filled contour of the analytical temperature distribution
pyplot.figure(figsize=(7.0, 6.0))
pyplot.xlabel('x [m]')
pyplot.ylabel('y [m]')
levels = numpy.linspace(0.0, 80.0, num=101)
contf = pyplot.contourf(x, y, T_analytical, levels=levels)
cbar = pyplot.colorbar(contf)
cbar.set_label('Temperature [°C]')
pyplot.axis('scaled', adjustable='box');

Plot the numerical temperature distribution from the `ftcs_JIT` code. Be sure, that the `levels` in this plot are identical to the one in the analytical solution plot above to allow a fair comparison.

In [ ]:
# Plot the filled contour of the temperature distribution from the ftcs_JIT code
pyplot.figure(figsize=(7.0, 6.0))
pyplot.xlabel('x [m]')
pyplot.ylabel('y [m]')
levels = numpy.linspace(0.0, 80.0, num=101)
contf = pyplot.contourf(x, y, T, levels=levels)
cbar = pyplot.colorbar(contf)
cbar.set_label('Temperature [°C]')
pyplot.axis('scaled', adjustable='box');

Plot the difference between numerical and analytical solution. You have to adapt the `levels` to the temperature difference between numerical and analytical solution.

In [ ]:
# Plot the filled contour of the temperature distribution from the ftcs_JIT code
pyplot.figure(figsize=(7.0, 6.0))
pyplot.xlabel('x [m]')
pyplot.ylabel('y [m]')
levels = numpy.linspace(-1e-1, 1e-1, num=101)
contf = pyplot.contourf(x, y, T-T_analytical, levels=levels)
cbar = pyplot.colorbar(contf)
cbar.set_label(r'$T_{FD} - T_{analytical}$ [°C]')
pyplot.axis('scaled', adjustable='box');

Discuss the error distribution. Where are large and small errors? Explain why.

## What we learned:

* How to estimate the accuracy of the FTCS code to solve the 2D heat equation by validating the numerical against an analytical solution